In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from lstm_class import Lstm
from sklearn.metrics import mean_squared_error

# Load Dataset

In [8]:
std_stocks = pd.read_csv("../data/stock_standard_1.csv").iloc[:, 3:] # Drop the First Three Columns
std_stocks = std_stocks.sort_values(by=['tic', 'datadate'])
std_stocks.drop(columns=['conm', 'gsector'], inplace=True)
std_stocks.reset_index(inplace=True)
stock_names = std_stocks['tic'].unique()
std_stocks.head()

,index,datadate,tic,cshtrm,prccm,prchm,prclm,trt1m,CAPEI,evm,...,high_low_ratio,RSI_14,MA_3,price_to_MA_3,return_momentum_6m,MA_6,return_momentum_9m,MA_9,return_momentum_12m,MA_12
0,22,2011-03-31,AMZN,0.057669,0.050441,0.047726,0.048270,0.449252,0.657229,0.104348,...,0.020950,0.599129,0.050675,0.673276,0.278596,0.050655,0.346955,0.046881,0.203680,0.044979
1,64,2011-04-30,AMZN,0.055855,0.054863,0.052030,0.052741,0.479247,0.657454,0.104348,...,0.020388,0.647066,0.053232,0.698572,0.291211,0.052154,0.350194,0.049443,0.222852,0.046454
2,106,2011-05-31,AMZN,0.051408,0.055111,0.054307,0.057432,0.427200,0.657571,0.104227,...,0.010842,0.669937,0.055518,0.671418,0.270170,0.053198,0.327768,0.051806,0.249207,0.048244
3,148,2011-06-30,AMZN,0.046094,0.057310,0.054270,0.054622,0.449368,0.657700,0.104227,...,0.022005,0.668990,0.057899,0.669438,0.275018,0.054399,0.255807,0.053366,0.306600,0.050637
4,190,2011-07-31,AMZN,0.044648,0.062395,0.059825,0.061283,0.479955,0.657969,0.104227,...,0.017924,0.721217,0.060509,0.699107,0.333047,0.056992,0.267572,0.055250,0.309609,0.053266


In [ ]:
X = []
y = []

In [ ]:
unique_stocks = df['tic'].unique()
unique_stocks

In [ ]:
df = norm_stocks.sort_values(by=['tic', 'datadate'])

X = []
y = []

unique_stocks = df[stock_id_col].unique()

for stock in unique_stocks:
    stock_data = df[df[stock_id_col] == stock]
    stock_features = stock_data[feature_cols].values
    stock_targets = stock_data[target_col].values

    for i in range(len(stock_data) - sequence_length):
        X.append(stock_features[i:i + sequence_length])
        y.append(stock_targets[i + sequence_length])

X = np.array(X)
y = np.array(y)

# Prepare Data for each asset

In [ ]:
# Create Sequential Data
def create_sequential_data(data, sequence_length = 7):
    X_sequential, y_sequential = [], []
    for i in range(len(data) - sequence_length):
        X_sequential.append(data[i:i+sequence_length])
        y_sequential.append(data[i+sequence_length])
        return np.array(X_sequential), np.array(y_sequential)
    
X, y = create_sequential_data(returns_data_scaled, sequence_length=7)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
train_size = int(len(returns_data_scaled) * 0.7)
X_train, y_test = X[:train_size], y[train_size:]
X_test, y_test = X[train_size:], y[]

In [ ]:
# Split the data into train and test sets (70% train, 30% test)
train_size = int(len(returns_data_scaled) * 0.7)
train, test = returns_data_scaled[:train_size], returns_data_scaled[train_size:]

# Function to create sequences
def create_sequences(data, sequence_length=7):
    X, y = [], []
    for i in range(len(data) - sequence_length - 1):
        X.append(data[i:(i + sequence_length), 0])
        y.append(data[i + sequence_length, 0])
    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = create_sequences(train, sequence_length=7)
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

In [ ]:
X_train[:, 0]

In [ ]:
X_train[]

In [ ]:
# Train Models for each Stock
models = {}
sequence_length = 7 # 70 Day Return Data
for i in range(num_stocks):
    stock_data = train[:, i].reshape(-1, 1) # Convert 1D array to 2D Vector
    lstm_model = Lstm(input_shape=(sequence_length, 1))
    X_train, y_train = create_sequences(stock_data, sequence_length)
    print(f"Shape of X_train: {X_train.shape}")
    print(f"Shape of y_train: {y_train.shape}")
    lstm_model.fit(X_train, y_train)
    models[i] = lstm_model

In [ ]:
# Store MSE
mse = {}
# Function to predict returns
def predict_returns(models, data, sequence_length=7):
    predicted_returns = []
    for i in range(num_stocks):
        model = models[i]
        stock_data = data[:, i].reshape(-1, 1)
        X_test, y_test = create_sequences(stock_data, sequence_length)
        #print(f"For Stock {i + 1} Shape of X_test: {X_test.shape}")
        #print(f"For Stock {i + 1} Shape of y_test: {y_test.shape}")
        predicted_return = model.model.predict(X_test)
        #print(f"For Stock {i + 1} Shape of predicted_return: {predicted_return.shape}")
        mse_ = mean_squared_error(y_test, predicted_return.flatten())
        print(f"For Stock {i + 1} the MSE is {mse_}")
        mse[i] = mse_
        predicted_returns.append(predicted_return.flatten())
    return np.array(predicted_returns).T

# Predict returns using the LSTM models
predicted_returns = predict_returns(models, test, sequence_length)
# Rescale the predicted returns to original scale
#predicted_returns = scaler.inverse_transform(predicted_returns)
print(mse)

In [ ]:
predicted_returns[:, 0]

In [ ]:
test[7:, :][:, 0]

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot((1 + test[7:, :][:, 0]).cumprod(), label='Actual Returns')
plt.plot((1 + predicted_returns[:, 0]).cumprod(), label='Predicted Returns')
plt.title(f'Predicted vs Actual Returns for Stock 1')
plt.xlabel('Days')
plt.ylabel('Returns')
plt.legend()
plt.show()

In [ ]:
# Plot predicted vs actual returns for a single stock
stock_index = 1  # Change this index to plot for a different stock
actual_returns = scaler.inverse_transform(test[sequence_length:])[:, stock_index]

plt.figure(figsize=(12, 6))
plt.plot(actual_returns, label='Actual Returns')
plt.plot(predicted_returns[:, stock_index], label='Predicted Returns')
plt.title(f'Predicted vs Actual Returns for Stock {stock_index + 1}')
plt.xlabel('Days')
plt.ylabel('Returns')
plt.legend()
plt.show()

In [ ]:
100 * (1 + actual_returns).cumprod()

In [ ]:
100 * (1 + predicted_returns[:, stock_index]).cumprod()

In [ ]:
# Plot the actual vs predicted prices for the first asset
plt.figure(figsize=(12, 6))
plt.plot(actual_returns, label='Actual Returns')
plt.plot(predicted_returns[:, stock_index], label='Predicted Returns')
plt.title('Actual vs Predicted Returns of the First Asset')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Plot the actual vs predicted prices for the first asset
plt.figure(figsize=(12, 6))
plt.plot(100 * (1 + actual_returns).cumprod(), label='Actual Prices')
plt.plot(100 * (1 + predicted_returns[:, stock_index]).cumprod(), label='Predicted Prices')
plt.title('Actual vs Predicted Prices of the First Asset')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.show()